# Employee Churn

We got employee data from a few companies. We have data about all employees who joined from 2011/01/24 to 2015/12/13. For each employee, we also know if they are still at the company as of 2015/12/13 or they have quit. Beside that, we have general info about the employee, such as avg salary during her tenure, dept, and yrs of experience.

As said above, the goal is to predict employee retention and understand its main drivers. Specifically, you should:


1. Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate employee headcount, for each company on each day, from 2011/01/24 to 2015/12/13. That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have already quit, then company headcount on 2012/03/02 for company 1 would be 1000. You should create a table with 3 columns: day, employee_headcount, company_id


- What are the main factors that drive employee churn? Do they make sense? Explain your findings


- If you could add to this data set just one variable that could help explain employee churn, what would that be?


### Data Checking

In [21]:
import pandas as pd
import numpy as np
pandas.set_option('display.max_columns', 10)
pandas.set_option('display.width', 350)
  
#read from google drive
data=pandas.read_csv(".\employee_retention.csv")
  
print(data.head())

   employee_id  company_id              dept  seniority    salary   join_date   quit_date
0      13021.0           7  customer_service         28   89000.0  2014-03-24  2015-10-30
1     825355.0           7         marketing         20  183000.0  2013-04-29  2014-04-04
2     927315.0           4         marketing         14  101000.0  2014-10-13         NaN
3     662910.0           7  customer_service         20  115000.0  2012-05-14  2013-06-07
4     256971.0           2      data_science         23  276000.0  2011-10-17  2014-08-22


In [2]:
# explore data
data.shape

(24702, 7)

In [26]:
data['company_id'].unique()
# only 12 companies in total - company 1 to company 12

array([ 7,  4,  2,  9,  1,  6, 10,  5,  3,  8, 11, 12], dtype=int64)

In [16]:
# to check how many NaN for quit date field - meaning they are still with the company (for heardcount calculation)
data['quit_date'].isnull().sum()

11192

### Create table with employee headcount by day

In [88]:
# create the dates from 2011/01/24 to 2015/12/13
dates = pd.date_range('2011-01-24', '2015-12-13')
dates = pd.DataFrame({'dates': dates})
dates

,dates
0,2011-01-24
1,2011-01-25
2,2011-01-26
3,2011-01-27
4,2011-01-28
...,...
1780,2015-12-09
1781,2015-12-10
1782,2015-12-11
1783,2015-12-12


In [31]:
# subset the company 1 part first
data_cpn1 = data[data['company_id']==1]
data_cpn1.shape

(8486, 7)

In [32]:
data_cpn1.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
9,504159.0,1,sales,7,104000.0,2012-06-12,NaN
12,939058.0,1,marketing,1,48000.0,2012-12-10,2013-11-15
15,265226.0,1,data_science,4,80000.0,2014-05-27,2015-07-10
19,982668.0,1,customer_service,14,76000.0,2015-07-27,NaN
28,888375.0,1,customer_service,5,75000.0,2011-08-01,2013-10-04


In [56]:
join_dates = data_cpn1['join_date']
quit_dates = data_cpn1['quit_date']

In [63]:
join_dates.value_counts()

2011-11-28    44
2014-10-27    42
2013-12-16    40
2012-01-03    40
2014-09-22    40
              ..
2015-04-21     1
2011-12-28     1
2014-01-16     1
2011-09-20     1
2014-10-28     1
Name: join_date, Length: 844, dtype: int64

In [92]:
join_date_ct = pd.DataFrame({'join_date': join_dates.value_counts().index,
                'join_date_ct': join_dates.value_counts().values})
#! need to use pd.to_datetime to convert the datetime in the original data frim object to datetime 
# to be able to merge with the datetime range i created
join_date_ct['join_date'] = pd.to_datetime(join_date_ct['join_date']) 
join_date_ct

,join_date,join_date_ct
0,2011-11-28,44
1,2014-10-27,42
2,2013-12-16,40
3,2012-01-03,40
4,2014-09-22,40
...,...,...
839,2015-04-21,1
840,2011-12-28,1
841,2014-01-16,1
842,2011-09-20,1


In [91]:
quit_date_ct = pd.DataFrame({'quit_date': quit_dates.value_counts().index,
                'quit_date_ct': quit_dates.value_counts().values})
#! need to use pd.to_datetime to convert the datetime in the original data frim object to datetime 
# to be able to merge with the datetime range i created
quit_date_ct['quit_date'] = pd.to_datetime(quit_date_ct['quit_date']) 
quit_date_ct

,quit_date,quit_date_ct
0,2015-05-08,38
1,2015-05-29,37
2,2015-10-09,37
3,2015-02-27,36
4,2015-06-26,36
...,...,...
433,2014-09-25,1
434,2012-02-29,1
435,2015-05-07,1
436,2015-10-19,1


In [93]:
dates_headct = pd.merge(dates, join_date_ct, how = 'left', left_on = 'dates', right_on = 'join_date')

In [95]:
dates_headct = pd.merge(dates_headct, quit_date_ct, how = 'left', left_on = 'dates', right_on = 'quit_date')

In [108]:
dates_headct
# hmmmm, where are the quit date values

,dates,join_date,join_date_ct,quit_date,quit_date_ct
0,2011-01-24,2011-01-24,25.0,NaT,NaN
1,2011-01-25,2011-01-25,2.0,NaT,NaN
2,2011-01-26,2011-01-26,2.0,NaT,NaN
3,2011-01-27,NaT,NaN,NaT,NaN
4,2011-01-28,NaT,NaN,NaT,NaN
...,...,...,...,...,...
1780,2015-12-09,2015-12-09,1.0,NaT,NaN
1781,2015-12-10,2015-12-10,1.0,NaT,NaN
1782,2015-12-11,NaT,NaN,NaT,NaN
1783,2015-12-12,NaT,NaN,NaT,NaN


In [105]:
# check to confirm both join date and quit date data sets are joined properly
dates_headct.loc[dates_headct['quit_date'].notnull(),]

,dates,join_date,join_date_ct,quit_date,quit_date_ct
270,2011-10-21,NaT,NaN,2011-10-21,1.0
291,2011-11-11,NaT,NaN,2011-11-11,1.0
302,2011-11-22,2011-11-22,1.0,2011-11-22,1.0
305,2011-11-25,NaT,NaN,2011-11-25,1.0
319,2011-12-09,NaT,NaN,2011-12-09,1.0
...,...,...,...,...,...
1765,2015-11-24,2015-11-24,2.0,2015-11-24,2.0
1768,2015-11-27,NaT,NaN,2015-11-27,35.0
1774,2015-12-03,2015-12-03,1.0,2015-12-03,1.0
1775,2015-12-04,NaT,NaN,2015-12-04,26.0


In [119]:
# fill NaNs with 0 in join_date_ct and quit_date_ct to work out the cumulative counts
dates_headct['join_date_ct'] = dates_headct['join_date_ct'].fillna(0)
dates_headct['quit_date_ct'] = dates_headct['quit_date_ct'].fillna(0)

In [120]:
# count accumulated joined number
dates_headct['join_cum_count'] = dates_headct['join_date_ct'].cumsum()
dates_headct['quit_cum_count'] = dates_headct['quit_date_ct'].cumsum()
dates_headct

,dates,join_date,join_date_ct,quit_date,quit_date_ct,join_cum_count,quit_cum_count
0,2011-01-24,2011-01-24,25.0,NaT,0.0,25.0,0.0
1,2011-01-25,2011-01-25,2.0,NaT,0.0,27.0,0.0
2,2011-01-26,2011-01-26,2.0,NaT,0.0,29.0,0.0
3,2011-01-27,NaT,0.0,NaT,0.0,29.0,0.0
4,2011-01-28,NaT,0.0,NaT,0.0,29.0,0.0
...,...,...,...,...,...,...,...
1780,2015-12-09,2015-12-09,1.0,NaT,0.0,8485.0,4621.0
1781,2015-12-10,2015-12-10,1.0,NaT,0.0,8486.0,4621.0
1782,2015-12-11,NaT,0.0,NaT,0.0,8486.0,4621.0
1783,2015-12-12,NaT,0.0,NaT,0.0,8486.0,4621.0


In [121]:
# check the data but take the ones with quit figures to have a sense
dates_headct.loc[dates_headct['quit_date'].notnull(),]

,dates,join_date,join_date_ct,quit_date,quit_date_ct,join_cum_count,quit_cum_count
270,2011-10-21,NaT,0.0,2011-10-21,1.0,1306.0,1.0
291,2011-11-11,NaT,0.0,2011-11-11,1.0,1402.0,2.0
302,2011-11-22,2011-11-22,1.0,2011-11-22,1.0,1452.0,3.0
305,2011-11-25,NaT,0.0,2011-11-25,1.0,1454.0,4.0
319,2011-12-09,NaT,0.0,2011-12-09,1.0,1527.0,5.0
...,...,...,...,...,...,...,...
1765,2015-11-24,2015-11-24,2.0,2015-11-24,2.0,8425.0,4558.0
1768,2015-11-27,NaT,0.0,2015-11-27,35.0,8431.0,4593.0
1774,2015-12-03,2015-12-03,1.0,2015-12-03,1.0,8461.0,4594.0
1775,2015-12-04,NaT,0.0,2015-12-04,26.0,8461.0,4620.0


In [126]:
dates_headct['headcount'] = dates_headct['join_cum_count'] - dates_headct['quit_cum_count']
# random check on row 300-320
dates_headct.iloc[300:320,]
# looks good!

,dates,join_date,join_date_ct,quit_date,quit_date_ct,join_cum_count,quit_cum_count,headcount
300,2011-11-20,NaT,0.0,NaT,0.0,1433.0,2.0,1431.0
301,2011-11-21,2011-11-21,18.0,NaT,0.0,1451.0,2.0,1449.0
302,2011-11-22,2011-11-22,1.0,2011-11-22,1.0,1452.0,3.0,1449.0
303,2011-11-23,2011-11-23,2.0,NaT,0.0,1454.0,3.0,1451.0
304,2011-11-24,NaT,0.0,NaT,0.0,1454.0,3.0,1451.0
305,2011-11-25,NaT,0.0,2011-11-25,1.0,1454.0,4.0,1450.0
306,2011-11-26,NaT,0.0,NaT,0.0,1454.0,4.0,1450.0
307,2011-11-27,NaT,0.0,NaT,0.0,1454.0,4.0,1450.0
308,2011-11-28,2011-11-28,44.0,NaT,0.0,1498.0,4.0,1494.0
309,2011-11-29,2011-11-29,1.0,NaT,0.0,1499.0,4.0,1495.0
